In [31]:
!pip install -q h5py

## 変更点


## 実験メモ

#### 12/26
* augumentationをするために、逐次マスクからcoordsに変換した方がいい
* 角度は、0と2piでズレがほとんどないのにlossが大きくなってしまうので、単位円上の座標が良さそう
* embeddingが無理？ embeddingは有限通りの入力をd_model次元で表現するものだから、従うためには有限通りの座標に落とし込む必要がある。となるとやはり角度が有利か。
* 座標でembeddingするには、どのパッチの上に乗っているかをidxで表して、それを埋め込むのが良さそう

>推論時は、Greedyアルゴリズムの場合、再帰的に最初の単語から最後の単語まで一つづつ予測していきます（コード）。具体的には最初の推論でBOS（文章の始まり）のみを含む文章をターゲットとして、2個目の単語を予測します。そして予測した単語を含めて再度モデルを実行し、３個目の単語を予測します。このときモデルはソースであるドイツ語の文章と１〜２個目の英単語から３個目を予測することになります。これを繰り返し、モデルがEOS（文章の終わり）を出力するまで繰り返します。何番目の単語を予測するかで文章の長さが変わるのでtgt_maskは文章の長さを同じになるように値を設定します。

https://qiita.com/simayan/items/ea8bc5df150f7890d0e7

In [28]:

# image_path = '../../../../create_synthe_2023/synthetic_flw/flw/白紫/000000.png'
# basename = image_path.split('/')[-1]
# mask_path = image_path.replace('/flw/', '/mask/').replace(f'/{basename}', f'_10c/{basename}').replace('png', 'h5')
# mask_path, basename

('../../../../create_synthe_2023/synthetic_flw/mask/白紫_10c/000000.h5',
 '000000.png')

In [1]:
exp_name = 'exp1'

In [2]:
!python -V

import torch
import torch.nn as nn
import torchvision
import segmentation_models_pytorch


print(f'{torch.__version__}')
print(f'{torchvision.__version__}')
print(f'{segmentation_models_pytorch.__version__}')

torch.cuda.is_available(), torch.cuda.device_count()

Python 3.10.13
2.1.2
0.16.2
0.3.3


(True, 8)

## research 

In [3]:
class DummyDecoder(nn.Module):
    def __init__(self, pad_id):
        super().__init__()
        self.pad_id = pad_id

    def get_attn_subsequent_mask(self, sz: int) -> torch.Tensor:
        """
        Generates an upper-triangular matrix of -inf, with zeros on the diag.
        """
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

    def forward(self, tgt_cptn: torch.Tensor):
        # create masks, then pass to decoder
        tgt_pad_mask = (tgt_cptn == self.pad_id)
        tgt_mask = self.get_attn_subsequent_mask(tgt_cptn.size()[1])
        tgt_mask = tgt_mask.to(tgt_cptn.device)
        
        return tgt_pad_mask, tgt_mask

# Sample data
pad_id = 0
tgt_cptn = torch.tensor([[1, 2, 3, 0, 0], [4, 5, 0, 0, 0]])

# Instantiate and run the dummy decoder
decoder = DummyDecoder(pad_id)
tgt_pad_mask, tgt_mask = decoder(tgt_cptn)

# Output the results
print("Target Padding Mask:")
print(tgt_pad_mask)
print("\nTarget Mask:")
print(tgt_mask)


Target Padding Mask:
tensor([[False, False, False,  True,  True],
        [False, False,  True,  True,  True]])

Target Mask:
tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])


***
## 本実装


In [4]:
import torchvision.models as models

# vit_b_16モデル呼び出し
model_vit = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

# get_model()
model_vit = models.get_model('vit_b_16', weights='DEFAULT')

model_vit.heads = torch.nn.Identity()
torch.save(model_vit.state_dict(), 'vit_b_224_16.pth')

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class CustomVisionTransformer(models.VisionTransformer):
    def __init__(self, *args, **kwargs):
        super(CustomVisionTransformer, self).__init__(*args, **kwargs)
        
        self.heads = torch.nn.Identity()
    
    def forward(self, x: torch.Tensor):
        # Reshape and permute the input tensor
        x = self._process_input(x)
        n = x.shape[0]

        return x

class CustomTransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
        super(CustomTransformerDecoderLayer, self).__init__()

        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim_feedforward, d_model)
        )

    def forward(self, tgt, memory, tgt_mask=None, tgt_pad_mask=None):
        """
        tgt: shape []
        """
        
        # Self attention with residual connection
        attn_output1, _ = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_pad_mask)
        tgt = self.norm1(tgt + self.dropout(attn_output1))
        
        # Cross attention with residual connection
        attn_output2, attn_weights = self.multihead_attn(tgt, memory, memory)
        tgt = self.norm2(tgt + self.dropout(attn_output2))
        
        # Feed forward with residual connection
        ff_output = self.feed_forward(tgt)
        tgt = tgt + self.dropout(ff_output)
        
        return tgt

class CustomTransformerDecoder(nn.Module):
    def __init__(self, d_model, nhead, num_decoder_layers, target_vocab_size, pad_id=0):
        super(CustomTransformerDecoder, self).__init__()

        self.embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.decoder_layers = nn.ModuleList([CustomTransformerDecoderLayer(d_model, nhead) for _ in range(num_decoder_layers)])
        self.pad_id = pad_id

    def get_attn_subsequent_mask(self, sz: int) -> torch.Tensor:
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

    def forward(self, tgt, memory):
        tgt_pad_mask = (tgt == self.pad_id)
        tgt = self.embedding(tgt) # tgt_after_embedding: [batch, target_len, d_model]
        tgt = self.pos_encoder(tgt)
        
        # create masks, then pass to decoder
        batch_size, target_len, d_model = tgt.shape
        tgt_mask = self.get_attn_subsequent_mask(target_len)

        tgt_mask = tgt_mask.to(tgt.device)

        for i, layer in enumerate(self.decoder_layers):
            tgt = layer(tgt, memory, tgt_mask=tgt_mask, tgt_pad_mask=tgt_pad_mask)

        # [batch, target_len?, d_model]
        return tgt

class CoordsViT(nn.Module):
    def __init__(self, image_size=224, patch_size=16, d_model=768, nhead=12, num_encoder_layers=12, num_decoder_layers=12, target_vocab_size=11, channels=3, out_channel=2):
        super(CoordsViT, self).__init__()
        
        # Encoder
        self.encoder = CustomVisionTransformer(
            image_size=image_size,
            patch_size=patch_size,
            num_layers=num_encoder_layers,
            num_heads=nhead,
            hidden_dim=d_model,
            mlp_dim=3072
        )
        self.encoder.load_state_dict(torch.load('vit_b_224_16.pth'))
        
        # Decoder
        self.decoder = CustomTransformerDecoder(d_model, nhead, num_decoder_layers, target_vocab_size)

        # Coords Head
        self.coords_heads = nn.Linear(d_model, out_channel)

    def forward(self, img, tgt):
        """
        tgt: [batch, target_len]
        """
        # Encode image
        encoder_output = self.encoder(img) # encoder_output: torch.Size([seq_len, batch, d_model])
        
        # Decode to get the target sequence
        tgt_out = self.decoder(tgt, encoder_output)  # tgt_out: [batch, target_len?, d_model]

        output = self.coords_heads(tgt_out)  # output: [batch, target_len?, 2], yx coords is 2d
        
        return output


In [19]:
import os
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from glob import glob

from multi_channel_img_io import read_img, write_img

image_size = 224

class OneHotTensorEncode(object):
    def __init__(self, n_classes=10):
        self.n_classes = n_classes

    def __call__(self, mask, debug=False):
        """
        Convert a 1-channel grayscale PIL image with n_classes unique values 
        into an n_classes-channel one-hot encoded tensor.
        """

        mask = np.array(mask)
        one_hot = torch.zeros(self.n_classes, mask.shape[0], mask.shape[1], dtype=torch.float32)
        for i in range(self.n_classes):
            one_hot[i, :, :] = torch.tensor(mask == i, dtype=torch.float32)
            if debug:
                count = np.sum(mask == i)
                print(f"Number of pixels with value {i}: {count}")
            
                plt.imshow(one_hot[i, :, :].numpy(), cmap='gray')
                plt.title(f"Class {i} One-hot Encoded Channel")
                plt.colorbar()
                plt.show()
        return one_hot

class RandomTransformsDual:
    """
    画像とマスクの両方にランダム変換を適用するクラス
    """
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, mask):
        seed = np.random.randint(2147483647)  # 大きな数でランダムシードを設定
        np.random.seed(seed)  # 同じシードを使用して、画像とマスクの両方に同じ変換を確実に適用する

        for t in self.transforms:
            img = t(img)

        np.random.seed(seed)
        for t in self.transforms:
            mask = t(mask)

        return img, mask
    
def custom_crop(image, trim_margin=180):
    return image.crop((trim_margin, trim_margin, image.width - trim_margin, image.height - trim_margin))

def channelize(mask_array, n_classes=10):
    mask_10d = torch.zeros(n_classes, mask_array.shape[0], mask_array.shape[1], dtype=torch.float32)
    for i in range(n_classes): 
        mask_10d[i, :, :] = torch.tensor(mask == i, dtype=torch.float32)
    return mask_10d

def get_center_array(mask_10d, coord_filter, relative=False):
    coords = []
    for mask_i in mask_10d[1:]:
        if int(mask_i.sum()) == 0:
            coords.append([0, 0])
            continue
        center_yx = [int((mask_i * coord_filter[:, :, 0]).sum() / mask_i.sum()), int((mask_i * coord_filter[:, :, 1]).sum() / mask_i.sum())]
        if relative:
            center_yx[0] /= mask_i.shape[0]
            center_yx[1] /= mask_i.shape[1]
        coords.append(center_yx)
    assert len(coords) == len(mask_10d) - 1, 'dim incorrect'
    return torch.Tensor(coords)

class SegmentationDataset(Dataset):
    def __init__(self, image_glob_path, image_size, image_transforms=None, mask_transforms=None, random_transforms=None):
        self.image_glob_path = image_glob_path
        self.image_transforms = image_transforms
        self.mask_transforms = mask_transforms
        self.random_transforms = random_transforms
        self.image_paths = sorted(glob(image_glob_path))
        self.coord_filter = np.array([[(i, j) for j in range(image_size)] for i in range(image_size)])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        basename = image_path.split('/')[-1]
        mask_path = image_path.replace('/flw/', '/mask/').replace(f'/{basename}', f'_10c/{basename}').replace('png', 'h5')
        image = Image.open(image_path).convert("RGB")
        mask = read_img(mask_path)
        print(mask.shape, mask_path)
        mask = Image.fromarray(mask)

        if self.random_transforms:
            image, mask = self.random_transforms(image, mask)

        if self.image_transforms:
            image = self.image_transforms(image)

        if self.mask_transforms:
            mask = self.mask_transforms(mask)
            coords = get_center_array(mask, self.coord_filter)

        return image, mask

random_transforms = RandomTransformsDual([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(180)  # ±180度のランダムな回転
])

image_transforms = transforms.Compose([
    transforms.Lambda(custom_crop),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

mask_transforms = transforms.Compose([
    transforms.Lambda(custom_crop),
    transforms.Resize((image_size, image_size)),
    # OneHotTensorEncode(n_classes=10),
])

flw_glob_path = '../../../../create_synthe_2023/synthetic_flw/flw/*/*'

# image_glob_path, mask_glob_path, image_transforms=None, mask_transforms=None, random_transforms=None):
dataset = SegmentationDataset(flw_glob_path, image_size, image_transforms=image_transforms, mask_transforms=mask_transforms)
dataset_size = len(dataset)

train_ratio = 0.7
val_ratio = 0.3

train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
# test_size = dataset_size - train_size - val_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# データローダーの作成
batch_size = 16
num_workers = 1
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [20]:
%%time

for c, i in val_dataloader:
    break

(756, 756, 9) ../../../../create_synthe_2023/synthetic_flw/mask/薄黄色_10c/064893.h5


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/PIL/Image.py", line 3070, in fromarray
    mode, rawmode = _fromarray_typemap[typekey]
KeyError: ((1, 1, 9), '|u1')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 364, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataset.py", line 364, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx 

(1436, 1436, 9) ../../../../create_synthe_2023/synthetic_flw/mask/白紫_10c/009705.h5
(756, 756, 10) ../../../../create_synthe_2023/synthetic_flw/mask/薄黄色_10c/050310.h5


In [15]:
%%time

for c, i in val_dataloader:
    break

CPU times: user 166 ms, sys: 2.75 s, total: 2.91 s
Wall time: 12 s


In [17]:
i.shape

torch.Size([16, 10, 224, 224])

In [ ]:
import math
import torch
import torch.nn as nn

from tqdm import tqdm
import segmentation_models_pytorch.utils as smp_utils

patch_size = 16
num_patch_1d = (image_size // patch_size)
indexer_1d = np.array([num_patch_1d, 1])

d_model = 256
nhead = 8
num_encoder_layers = 3
num_decoder_layers = 3
target_vocab_size = num_patch_1d ** 2 # patchの数を語彙数とする
# len_out_seq = 9

model = CoordsViT(target_vocab_size=target_vocab_size)

for i, (images, coords) in enumerate(train_dataloader):
    coords_idx_2d = coords // num_patch_1d
    coords_idx_1d = coords_idx_2d @ indexer_1d
    target_seq = coords_idx_1d.long()
    break

output = model(images, target_seq)

print(f'target_seq.shape: {target_seq.shape}')
print(f'output.shape: {output.shape}')

In [ ]:
# device = torch.device("cpu")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}, Train") as pbar_train:
        for i, (images, coords) in enumerate(train_dataloader):
            images = images.to(device)
            coords_idx_2d = coords // num_patch_1d
            coords_idx_1d = coords_idx_2d @ indexer_1d
            target_seq = coords_idx_1d.long().to(device)

            output = model(images, target_seq)
            loss = criterion(output, coords.to(device))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar_train.set_postfix({"Train Loss": loss.item()})
            pbar_train.update(1)

    save_path = f'./models/{exp_name}_epoch{epoch + 1}'
    torch.save(model.state_dict(), save_path)

    model.eval()
    with tqdm(total=len(val_dataloader), desc=f"Epoch {epoch + 1} / {num_epochs}, Val") as pbar_val:
        for i, (images, coords) in enumerate(val_dataloader):
            images = images.to(device)
            coords_idx_2d = coords // num_patch_1d
            coords_idx_1d = coords_idx_2d @ indexer_1d
            target_seq = coords_idx_1d.long().to(device)

            output = model(images, target_seq)
            loss = criterion(output, coords)

            pbar_val.set_postfix({"Val Loss": loss.item()})
            pbar_val.update(1)


Epoch 1/20, Train:   7%|▋         | 1498/21875 [04:28<47:03,  7.22it/s, Train Loss=3.24e+3]  

In [ ]:
for channel in cross_attention_maps.to('cpu')[0]:
    print(channel)
    break

In [ ]:
for channel in mask_proportions.to('cpu')[0]:
    print(channel)
    break

In [ ]:
for i in 

In [11]:
predicted_indices = torch.argmax(pred_probs, dim=-1)
predicted_indices

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10]], device='cuda:2')

In [25]:
loss_mask

tensor(2.9710, device='cuda:2', grad_fn=<AddBackward0>)

In [26]:
loss_seq

tensor(0.2592, device='cuda:2', grad_fn=<BinaryCrossEntropyBackward>)

In [23]:
predicted_indices = torch.argmax(pred_probs, dim=-1)
predicted_indices

tensor([[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
        [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]], device='cuda:2')

In [17]:
loss_seq

tensor(0.2662, device='cuda:2', grad_fn=<BinaryCrossEntropyBackward>)

In [18]:
loss_mask

tensor(2.9672, device='cuda:2', grad_fn=<AddBackward0>)

In [19]:
loss_seq

tensor(0.7469, device='cuda:2')

In [13]:
import torch.nn.functional as F
print(predicted_indices.shape)
pred_one_hot = F.one_hot(predicted_indices).float()
gt_one_hot = F.one_hot(target_seq).float()
loss = F.binary_cross_entropy_with_logits(pred_one_hot, gt_one_hot)

print(loss)

torch.Size([12, 11])
tensor(0.7357)


In [9]:
target_seq

tensor([[ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5, 10, 10, 10, 10, 10],
        [ 0,  1,  2,  3,  4,  5,  6,  7, 10, 10, 10]])

In [ ]:
import torch
import matplotlib.pyplot as plt

def recreate_image_from_attention(cross_attention_map, img_size, patch_size):
    """
    cross_attention_map: shape [11, 1024] (or any [channel_out, flattened_img_size])
    img_size: Original image size (e.g., 224 for a 224x224 image)
    patch_size: The size of each patch (e.g., 16 for 16x16 patches)
    """
    channel_out, flattened_img_size = cross_attention_map.shape
    patches_per_dim = img_size // patch_size
    
    # Reshape to [channel_out, patches_per_dim, patches_per_dim]
    reshaped_attention_map = cross_attention_map.view(channel_out, patches_per_dim, patches_per_dim)
    
    # Initialize tensor to store the recreated image
    recreated_images = torch.zeros((channel_out, img_size, img_size))
    
    # Fill in the recreated image tensor
    for i in range(patches_per_dim):
        for j in range(patches_per_dim):
            recreated_images[:, i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] = reshaped_attention_map[:, i, j].unsqueeze(1).unsqueeze(2)
            
    return recreated_images


# Call the function
recreated_images = recreate_image_from_attention(cross_attention_maps[0][0], image_size, patch_size)

# Visualize
for i in range(recreated_images.shape[0]):
    plt.figure()
    plt.imshow(recreated_images[i].detach().numpy(), cmap="gray")
    plt.title(f"Channel {i}")
    plt.show()


In [ ]:
recreated_images = recreate_image_from_attention(mask_proportions[0], image_size, patch_size)

# Visualize
for i in range(recreated_images.shape[0]):
    plt.figure()
    plt.imshow(recreated_images[i].detach().numpy(), cmap="gray")
    plt.title(f"Channel {i}")
    plt.show()


## Idea 



* クラスラベルの順列を出力系列とするのは、あまりに難しそう。
* docoderに入力するのは、各パッチの特徴を並べたもの

target_seqの系列長と最終的に出力してほしいsequence_lengthは一般には同じである必要はありません。Transformerモデルの訓練時においては、通常、ターゲット系列（target_seq）は入力として使われるだけでなく、訓練のラベルとしても使用されます。その場合、target_seqの系列長は出力と一致するように設計されることが多いです。

しかし、推論（予測）時には、通常、開始トークンだけをtarget_seqとして使用し、モデルがその後のトークンを一つずつ生成するようにします。この場合、target_seqの初期の長さは1であり、出力のsequence_lengthは1から始まって任意の長さになり得ます（停止条件または最大長に達するまで）。

簡単に言うと、訓練と推論でtarget_seqの役割が少し異なるため、その長さもそれに応じて変わります。

In [52]:
predicted_indices = torch.argmax(output, dim=-1)
predicted_indices.shape == target_seq.shape

True